## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

May. 11th May. 18th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,May-15-2025,Condo/Co-op,9066 SW 73rd Ct #1502,Miami,FL,33156.0,327500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/9066-SW-73rd-C...,MARMLS,A11757523,N,Y,25.686531,-80.314424


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-05-12    31
2025-05-15    29
2025-05-16    26
2025-05-14    21
2025-05-13    20
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
75,PAST SALE,2025-05-12,Condo/Co-op,7012 Fisher Island Dr #7012,Miami Beach,FL,33109.0,16200000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,MARMLS,A11712235,N,Y,25.763482,-80.137699
80,PAST SALE,2025-05-15,Condo/Co-op,2811 S Bayshore Dr Unit 15A,Miami,FL,33133.0,12500000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11710639,N,Y,25.728751,-80.238013
7,PAST SALE,2025-05-16,Condo/Co-op,7953 Fisher Island Dr #7953,Fisher Island,FL,33109.0,5500000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,MARMLS,A11776077,N,Y,25.758915,-80.137511
27,PAST SALE,2025-05-16,Condo/Co-op,2627 S Bayshore Dr #1504,Miami,FL,33133.0,4050000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11776122,N,Y,25.732272,-80.234921
127,PAST SALE,2025-05-16,Condo/Co-op,1300 Monad Ter Unit 9B,Miami Beach,FL,33139.0,4000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11661491,N,Y,25.784161,-80.142633
55,PAST SALE,2025-05-15,Condo/Co-op,10201 Collins Ave #511,Bal Harbour,FL,33154.0,3750000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11754086,N,Y,25.895168,-80.123150
39,PAST SALE,2025-05-15,Condo/Co-op,100 S Pointe Dr #1602,Miami Beach,FL,33139.0,3260902.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11711551,N,Y,25.767180,-80.132822
17,PAST SALE,2025-05-12,Condo/Co-op,300 Collins Ave Unit 4D,Miami Beach,FL,33139.0,3099000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11751817,N,Y,25.772642,-80.133687
109,PAST SALE,2025-05-14,Condo/Co-op,8701 Collins Ave #306,Miami Beach,FL,33154.0,2850000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11664216,N,Y,25.872197,-80.121210
64,PAST SALE,2025-05-15,Condo/Co-op,10201 E Bay Harbor Dr #501,Bay Harbor Islands,FL,33154.0,2450000.0,4.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11742820,N,Y,25.892681,-80.130898


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-05-12,Condo/Co-op,7012 Fisher Island Dr #7012,Miami Beach,FL,33109.0,16200000.0,4.0,4.5,...,-80.137699,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,Dora Puig,Luxe Living Realty,None,None,Fredrik Eklund,Douglas Elliman,None,None
1,PAST SALE,2025-05-15,Condo/Co-op,2811 S Bayshore Dr Unit 15A,Miami,FL,33133.0,12500000.0,4.0,5.5,...,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Jessica Adams,Douglas Elliman,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None
2,PAST SALE,2025-05-16,Condo/Co-op,7953 Fisher Island Dr #7953,Fisher Island,FL,33109.0,5500000.0,2.0,2.5,...,-80.137511,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,Lilian Carrion,"Compass Florida, LLC",None,None,Nataly Stuart,"Socialite Realty, LLC",None,None
3,PAST SALE,2025-05-16,Condo/Co-op,2627 S Bayshore Dr #1504,Miami,FL,33133.0,4050000.0,3.0,3.5,...,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,Christina Fernandez,"ACC Realty, Inc.",None,None
4,PAST SALE,2025-05-16,Condo/Co-op,1300 Monad Ter Unit 9B,Miami Beach,FL,33139.0,4000000.0,3.0,3.5,...,-80.142633,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Todd Nordstrom,"Compass Florida, LLC.",Julian Fernandez,"Compass Florida, LLC.",Juan Zuluaga,Coldwell Banker Realty,None,None
5,PAST SALE,2025-05-15,Condo/Co-op,10201 Collins Ave #511,Bal Harbour,FL,33154.0,3750000.0,2.0,2.0,...,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,None,None,None,None,None,None,None,None
6,PAST SALE,2025-05-15,Condo/Co-op,100 S Pointe Dr #1602,Miami Beach,FL,33139.0,3260902.0,1.0,1.5,...,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Luis Paredes,Million Realty LLC,None,None,Matthew Lill,The Corcoran Group,None,None
7,PAST SALE,2025-05-12,Condo/Co-op,300 Collins Ave Unit 4D,Miami Beach,FL,33139.0,3099000.0,3.0,3.5,...,-80.133687,https://www.redfin.com/FL/Miami-Beach/300-Coll...,David Reid,One Sotheby's International Re,None,None,Anna Sherrill,One Sotheby's International Re,None,None
8,PAST SALE,2025-05-14,Condo/Co-op,8701 Collins Ave #306,Miami Beach,FL,33154.0,2850000.0,2.0,2.5,...,-80.121210,https://www.redfin.com/FL/Surfside/8701-Collin...,Miltiadis Kastanis,"Compass Florida, LLC.",None,None,Mikhail MudrIk,The Agency Florida LLC,None,None
9,PAST SALE,2025-05-15,Condo/Co-op,10201 E Bay Harbor Dr #501,Bay Harbor Islands,FL,33154.0,2450000.0,4.0,3.5,...,-80.130898,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,3009174 Romanello,"Compass Florida, LLC.",Claudia Villar,"Compass Florida, LLC.",3009174 Romanello,"Compass Florida, LLC.",Claudia Villar,"Compass Florida, LLC."


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

127


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$973,842


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$636


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$123,677,931


In [24]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [25]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
91
-----------
Input Previous Week Condo Average Sales Price:
1230121
-----------
Input Previous Week Condo Average PSF:
660
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
111940988.0


In [26]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,91,1230121,660,111940988.0


In [27]:
last_week['Last Week Total Dollar Volume'].iloc[0]

111940988.0

In [28]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          127              973842          636                    123677931.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [30]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [32]:
pd.set_option('display.max_columns',None)

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-12,Condo/Co-op,7012 Fisher Island Dr #7012,Miami Beach,FL,33109.0,16200000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4698.0,NaN,2016.0,NaN,3448.0,15489.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,MARMLS,A11712235,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,Dora Puig,Luxe Living Realty,None,None,Fredrik Eklund,Douglas Elliman,None,None,1,orange


In [34]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-12,Condo/Co-op,7012 Fisher Island Dr #7012,Miami Beach,FL,33109.0,16200000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4698.0,NaN,2016.0,NaN,3448.0,15489.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,MARMLS,A11712235,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,Dora Puig,Luxe Living Realty,None,None,Fredrik Eklund,Douglas Elliman,None,None,1,orange
1,PAST SALE,2025-05-15,Condo/Co-op,2811 S Bayshore Dr Unit 15A,Miami,FL,33133.0,12500000.0,4.0,5.5,C.G.B. SUBDIVISION,3247.0,NaN,2020.0,NaN,3850.0,7447.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11710639,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Jessica Adams,Douglas Elliman,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,2,blue
2,PAST SALE,2025-05-16,Condo/Co-op,7953 Fisher Island Dr #7953,Fisher Island,FL,33109.0,5500000.0,2.0,2.5,OCEANSIDE @ FISHER ISL CO,2420.0,NaN,1989.0,NaN,2273.0,7646.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,MARMLS,A11776077,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,Lilian Carrion,"Compass Florida, LLC",None,None,Nataly Stuart,"Socialite Realty, LLC",None,None,3,blue
3,PAST SALE,2025-05-16,Condo/Co-op,2627 S Bayshore Dr #1504,Miami,FL,33133.0,4050000.0,3.0,3.5,GROVENOR HOUSE CONDO,2640.0,NaN,2005.0,NaN,1534.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11776122,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,Christina Fernandez,"ACC Realty, Inc.",None,None,4,blue
4,PAST SALE,2025-05-16,Condo/Co-op,1300 Monad Ter Unit 9B,Miami Beach,FL,33139.0,4000000.0,3.0,3.5,ALTON BEACH REALTY CO SUB,2691.0,NaN,2021.0,NaN,1486.0,13370.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11661491,N,Y,25.784161,-80.142633,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Todd Nordstrom,"Compass Florida, LLC.",Julian Fernandez,"Compass Florida, LLC.",Juan Zuluaga,Coldwell Banker Realty,None,None,5,blue
5,PAST SALE,2025-05-15,Condo/Co-op,10201 Collins Ave #511,Bal Harbour,FL,33154.0,3750000.0,2.0,2.0,OCEANA BAL HARBOUR CONDO,1805.0,NaN,2016.0,NaN,2078.0,3864.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11754086,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,None,None,None,None,None,None,None,None,6,blue
6,PAST SALE,2025-05-15,Condo/Co-op,100 S Pointe Dr #1602,Miami Beach,FL,33139.0,3260902.0,1.0,1.5,CONTINUUM ON SOUTH BEACH,1201.0,NaN,2002.0,NaN,2715.0,3600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11711551,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Luis Paredes,Million Realty LLC,None,None,Matthew Lill,The Corcoran Group,None,None,7,blue
7,PAST SALE,2025-05-12,Condo/Co-op,300 Collins Ave Unit 4D,Miami Beach,FL,33139.0,3099000.0,3.0,3.5,300 COLLINS CONDO,1699.0,NaN,2018.0,NaN,1824.0,8986.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11751817,N,Y,25.772642,-80.133687,https://www.redfin.com/FL/Miami-Beach/300-Coll...,David Reid,One Sotheby's International Re,None,None,Anna Sherrill,One Sotheby's International Re,None,None,8,blue
8,PAST SALE,2025-05-14,Condo/Co-op,8701 Collins Ave #306,Miami Beach,FL,33154.0,2850000.0,2.0,2.5,8701 COLLINS AVE CONDO,1602.0,NaN,2019.0,NaN,1779.0,5903.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8

In [36]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-12,Condo/Co-op,7012 Fisher Island Dr #7012,Miami Beach,FL,33109.0,16200000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4698.0,NaN,2016.0,NaN,3448.0,15489.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,MARMLS,A11712235,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,Dora Puig,Luxe Living Realty,None,None,Fredrik Eklund,Douglas Elliman,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_05192025


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'PALAZZO DEL SOL/DELLA LUN at 7012 Fisher Island Dr #7012 in Miami Beach'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-12,Condo/Co-op,7012 Fisher Island Dr #7012,Miami Beach,FL,33109.0,16200000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4698.0,NaN,2016.0,NaN,3448.0,15489.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,MARMLS,A11712235,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,Dora Puig,Luxe Living Realty,None,None,Fredrik Eklund,Douglas Elliman,None,None,1,orange,PALAZZO DEL SOL/DELLA LUN at 7012 Fisher Islan...
1,PAST SALE,2025-05-15,Condo/Co-op,2811 S Bayshore Dr Unit 15A,Miami,FL,33133.0,12500000.0,4.0,5.5,C.G.B. SUBDIVISION,3247.0,NaN,2020.0,NaN,3850.0,7447.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11710639,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Jessica Adams,Douglas Elliman,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,2,blue,C.G.B. SUBDIVISION at 2811 S Bayshore Dr Unit ...
2,PAST SALE,2025-05-16,Condo/Co-op,7953 Fisher Island Dr #7953,Fisher Island,FL,33109.0,5500000.0,2.0,2.5,OCEANSIDE @ FISHER ISL CO,2420.0,NaN,1989.0,NaN,2273.0,7646.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,MARMLS,A11776077,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,Lilian Carrion,"Compass Florida, LLC",None,None,Nataly Stuart,"Socialite Realty, LLC",None,None,3,blue,OCEANSIDE @ FISHER ISL CO at 7953 Fisher Islan...
3,PAST SALE,2025-05-16,Condo/Co-op,2627 S Bayshore Dr #1504,Miami,FL,33133.0,4050000.0,3.0,3.5,GROVENOR HOUSE CONDO,2640.0,NaN,2005.0,NaN,1534.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11776122,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,Christina Fernandez,"ACC Realty, Inc.",None,None,4,blue,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #15...
4,PAST SALE,2025-05-16,Condo/Co-op,1300 Monad Ter Unit 9B,Miami Beach,FL,33139.0,4000000.0,3.0,3.5,ALTON BEACH REALTY CO SUB,2691.0,NaN,2021.0,NaN,1486.0,13370.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11661491,N,Y,25.784161,-80.142633,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Todd Nordstrom,"Compass Florida, LLC.",Julian Fernandez,"Compass Florida, LLC.",Juan Zuluaga,Coldwell Banker Realty,None,None,5,blue,ALTON BEACH REALTY CO SUB at 1300 Monad Ter Un...
5,PAST SALE,2025-05-15,Condo/Co-op,10201 Collins Ave #511,Bal Harbour,FL,33154.0,3750000.0,2.0,2.0,OCEANA BAL HARBOUR CONDO,1805.0,NaN,2016.0,NaN,2078.0,3864.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11754086,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,None,None,None,None,None,None,None,None,6,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
6,PAST SALE,2025-05-15,Condo/Co-op,100 S Pointe Dr #1602,Miami Beach,FL,33139.0,3260902.0,1.0,1.5,CONTINUUM ON SOUTH BEACH,1201.0,NaN,2002.0,NaN,2715.0,3600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11711551,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Luis Paredes,Million Realty LLC,None,None,Matthew Lill,The Corcoran Group,None,None,7,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
7,PAST SALE,2025-05-12,Condo/Co-op,300 Collins Ave Unit 4D,Miami Beach,FL,33139.0,3099000.0,3.0,3.5,300 COLLINS CONDO,1699.0,NaN,2018.0,NaN,1824.0,8986.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11751817,N,Y,25.772642,-80.133687,https://ww

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: PALAZZO DEL SOL/DELLA LUN closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,450,000 to $16,200,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Fisher Island, Miami Beach, Miami, Bal Harbour, Bay Harbor Islands, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 127 condo sales totaling $123,677,931 million from May. 11th to May. 18th. The previous week, brokers closed 91 condo sales totaling $111,940,988.

Last week’s units sold for an average of $973,842, lower than the $1,230,12

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-12,Condo/Co-op,7012 Fisher Island Dr #7012,Miami Beach,FL,33109.0,16200000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4698.0,NaN,2016.0,NaN,3448.0,15489.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,MARMLS,A11712235,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,Dora Puig,Luxe Living Realty,None,None,Fredrik Eklund,Douglas Elliman,None,None,1,orange,PALAZZO DEL SOL/DELLA LUN at 7012 Fisher Islan...
1,PAST SALE,2025-05-15,Condo/Co-op,2811 S Bayshore Dr Unit 15A,Miami,FL,33133.0,12500000.0,4.0,5.5,C.G.B. SUBDIVISION,3247.0,NaN,2020.0,NaN,3850.0,7447.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11710639,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Jessica Adams,Douglas Elliman,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,2,blue,C.G.B. SUBDIVISION at 2811 S Bayshore Dr Unit ...
2,PAST SALE,2025-05-16,Condo/Co-op,7953 Fisher Island Dr #7953,Fisher Island,FL,33109.0,5500000.0,2.0,2.5,OCEANSIDE @ FISHER ISL CO,2420.0,NaN,1989.0,NaN,2273.0,7646.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,MARMLS,A11776077,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,Lilian Carrion,"Compass Florida, LLC",None,None,Nataly Stuart,"Socialite Realty, LLC",None,None,3,blue,OCEANSIDE @ FISHER ISL CO at 7953 Fisher Islan...
3,PAST SALE,2025-05-16,Condo/Co-op,2627 S Bayshore Dr #1504,Miami,FL,33133.0,4050000.0,3.0,3.5,GROVENOR HOUSE CONDO,2640.0,NaN,2005.0,NaN,1534.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11776122,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,Christina Fernandez,"ACC Realty, Inc.",None,None,4,blue,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #15...
4,PAST SALE,2025-05-16,Condo/Co-op,1300 Monad Ter Unit 9B,Miami Beach,FL,33139.0,4000000.0,3.0,3.5,ALTON BEACH REALTY CO SUB,2691.0,NaN,2021.0,NaN,1486.0,13370.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11661491,N,Y,25.784161,-80.142633,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Todd Nordstrom,"Compass Florida, LLC.",Julian Fernandez,"Compass Florida, LLC.",Juan Zuluaga,Coldwell Banker Realty,None,None,5,blue,ALTON BEACH REALTY CO SUB at 1300 Monad Ter Un...
5,PAST SALE,2025-05-15,Condo/Co-op,10201 Collins Ave #511,Bal Harbour,FL,33154.0,3750000.0,2.0,2.0,OCEANA BAL HARBOUR CONDO,1805.0,NaN,2016.0,NaN,2078.0,3864.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11754086,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,None,None,None,None,None,None,None,None,6,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
6,PAST SALE,2025-05-15,Condo/Co-op,100 S Pointe Dr #1602,Miami Beach,FL,33139.0,3260902.0,1.0,1.5,CONTINUUM ON SOUTH BEACH,1201.0,NaN,2002.0,NaN,2715.0,3600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11711551,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Luis Paredes,Million Realty LLC,None,None,Matthew Lill,The Corcoran Group,None,None,7,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
7,PAST SALE,2025-05-12,Condo/Co-op,300 Collins Ave Unit 4D,Miami Beach,FL,33139.0,3099000.0,3.0,3.5,300 COLLINS CONDO,1699.0,NaN,2018.0,NaN,1824.0,8986.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11751817,N,Y,25.772642,-80.133687,https://ww

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: PALAZZO DEL SOL/DELLA LUN closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,450,000 to $16,200,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Fisher Island, Miami Beach, Miami, Bal Harbour, Bay Harbor Islands, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 127 condo sales totaling $123,677,931 million from May. 11th to May. 18th. The previous week, brokers closed 91 condo sales totaling $111,940,988.

Last week’s units sold for an average of $973,842, lower than the $1,230,12

In [62]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami-Beach/7012-Fisher-Island-Dr-33109/unit-7012/home/110132741


In [63]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/2811-S-Bayshore-Dr-33133/unit-15A/home/175175301


In [64]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Bay-Harbor-Islands/10201-E-Bay-Harbor-Dr-33154/unit-501/home/112999887


In [65]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Surfside/8701-Collins-Ave-33154/unit-306/home/168665606


In [66]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami/2811-S-Bayshore-Dr-33133/unit-15A/home/175175301


In [67]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Bay-Harbor-Islands/10201-E-Bay-Harbor-Dr-33154/unit-501/home/112999887


In [68]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-12,Condo/Co-op,7012 Fisher Island Dr #7012,Miami Beach,FL,33109.0,16200000.0,4.0,4.5,PALAZZO DEL SOL/DELLA LUN,4698.0,NaN,2016.0,NaN,3448.0,15489.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,MARMLS,A11712235,N,Y,25.763482,-80.137699,https://www.redfin.com/FL/Miami-Beach/7012-Fis...,Dora Puig,Luxe Living Realty,None,None,Fredrik Eklund,Douglas Elliman,None,None,1,orange,PALAZZO DEL SOL/DELLA LUN at 7012 Fisher Islan...
1,PAST SALE,2025-05-15,Condo/Co-op,2811 S Bayshore Dr Unit 15A,Miami,FL,33133.0,12500000.0,4.0,5.5,C.G.B. SUBDIVISION,3247.0,NaN,2020.0,NaN,3850.0,7447.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11710639,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Jessica Adams,Douglas Elliman,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,2,blue,C.G.B. SUBDIVISION at 2811 S Bayshore Dr Unit ...
2,PAST SALE,2025-05-16,Condo/Co-op,7953 Fisher Island Dr #7953,Fisher Island,FL,33109.0,5500000.0,2.0,2.5,OCEANSIDE @ FISHER ISL CO,2420.0,NaN,1989.0,NaN,2273.0,7646.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,MARMLS,A11776077,N,Y,25.758915,-80.137511,https://www.redfin.com/FL/Miami-Beach/7953-Fis...,Lilian Carrion,"Compass Florida, LLC",None,None,Nataly Stuart,"Socialite Realty, LLC",None,None,3,blue,OCEANSIDE @ FISHER ISL CO at 7953 Fisher Islan...
3,PAST SALE,2025-05-16,Condo/Co-op,2627 S Bayshore Dr #1504,Miami,FL,33133.0,4050000.0,3.0,3.5,GROVENOR HOUSE CONDO,2640.0,NaN,2005.0,NaN,1534.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11776122,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,Christina Fernandez,"ACC Realty, Inc.",None,None,4,blue,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #15...
4,PAST SALE,2025-05-16,Condo/Co-op,1300 Monad Ter Unit 9B,Miami Beach,FL,33139.0,4000000.0,3.0,3.5,ALTON BEACH REALTY CO SUB,2691.0,NaN,2021.0,NaN,1486.0,13370.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11661491,N,Y,25.784161,-80.142633,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Todd Nordstrom,"Compass Florida, LLC.",Julian Fernandez,"Compass Florida, LLC.",Juan Zuluaga,Coldwell Banker Realty,None,None,5,blue,ALTON BEACH REALTY CO SUB at 1300 Monad Ter Un...
5,PAST SALE,2025-05-15,Condo/Co-op,10201 Collins Ave #511,Bal Harbour,FL,33154.0,3750000.0,2.0,2.0,OCEANA BAL HARBOUR CONDO,1805.0,NaN,2016.0,NaN,2078.0,3864.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11754086,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,None,None,None,None,None,None,None,None,6,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
6,PAST SALE,2025-05-15,Condo/Co-op,100 S Pointe Dr #1602,Miami Beach,FL,33139.0,3260902.0,1.0,1.5,CONTINUUM ON SOUTH BEACH,1201.0,NaN,2002.0,NaN,2715.0,3600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11711551,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Luis Paredes,Million Realty LLC,None,None,Matthew Lill,The Corcoran Group,None,None,7,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
7,PAST SALE,2025-05-12,Condo/Co-op,300 Collins Ave Unit 4D,Miami Beach,FL,33139.0,3099000.0,3.0,3.5,300 COLLINS CONDO,1699.0,NaN,2018.0,NaN,1824.0,8986.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11751817,N,Y,25.772642,-80.133687,https://ww

## Time on Market Calculator

In [70]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 12, 23) ## List (Earlier) date
date2 = datetime(2025, 5, 15) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

143


## Clean CSV for Datawrapper Chart

In [71]:
chart_df = df_top_ten

In [72]:
chart_df = chart_df.fillna(" ")

In [73]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [74]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [75]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [76]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [77]:
chart_df['$/SQUARE FEET'].astype(float)

0    3448.0
1    3850.0
2    2273.0
3    1534.0
4    1486.0
5    2078.0
6    2715.0
7    1824.0
8    1779.0
9    1270.0
Name: $/SQUARE FEET, dtype: float64

In [78]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [79]:
csv_date_string = today.strftime("%m_%d_%Y")

In [80]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [81]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-05-12,7012 Fisher Island Dr #7012 in Miami Beach,"$16,200,000",4,4.5,"4,698",2016,"$3,448",MARMLS,A11712235,Dora Puig Luxe Living Realty,Fredrik Eklund Douglas Elliman
1,2025-05-15,2811 S Bayshore Dr Unit 15A in Miami,"$12,500,000",4,5.5,"3,247",2020,"$3,850",MARMLS,A11710639,Jessica Adams Douglas Elliman,NON-MLS MEMBER MAR NON MLS MEMBER
2,2025-05-16,7953 Fisher Island Dr #7953 in Fisher Island,"$5,500,000",2,2.5,"2,420",1989,"$2,273",MARMLS,A11776077,"Lilian Carrion Compass Florida, LLC","Nataly Stuart Socialite Realty, LLC"
3,2025-05-16,2627 S Bayshore Dr #1504 in Miami,"$4,050,000",3,3.5,"2,640",2005,"$1,534",MARMLS,A11776122,Ana Sanchez One Sotheby's International Realty...,"Christina Fernandez ACC Realty, Inc."
4,2025-05-16,1300 Monad Ter Unit 9B in Miami Beach,"$4,000,000",3,3.5,"2,691",2021,"$1,486",MARMLS,A11661491,"Todd Nordstrom Compass Florida, LLC. Julian Fe...",Juan Zuluaga Coldwell Banker Realty
5,2025-05-15,10201 Collins Ave #511 in Bal Harbour,"$3,750,000",2,2.0,"1,805",2016,"$2,078",MARMLS,A11754086,,
6,2025-05-15,100 S Pointe Dr #1602 in Miami Beach,"$3,260,902",1,1.5,"1,201",2002,"$2,715",MARMLS,A11711551,Luis Paredes Million Realty LLC,Matthew Lill The Corcoran Group
7,2025-05-12,300 Collins Ave Unit 4D in Miami Beach,"$3,099,000",3,3.5,"1,699",2018,"$1,824",MARMLS,A11751817,David Reid One Sotheby's International Re,Anna Sherrill One Sotheby's International Re
8,2025-05-14,8701 Collins Ave #306 in Miami Beach,"$2,850,000",2,2.5,"1,602",2019,"$1,779",MARMLS,A11664216,"Miltiadis Kastanis Compass Florida, LLC.",Mikhail MudrIk The Agency Florida LLC
9,2025-05-15,10201 E Bay Harbor Dr #501 in Bay Harbor Islands,"$2,450,000",4,3.5,"1,929",2016,"$1,270",MARMLS,A11742820,"3009174 Romanello Compass Florida, LLC. Claudi...","3009174 Romanello Compass Florida, LLC. Claudi..."
